In [1]:
import pandas as pd
from pyomo.environ import *

# Cargar los datos
df = pd.read_csv(r"C:\Users\Aminadab Cordova\Documents\MIAAD\DEA_problem\datos_talleres.csv")

# Lista de DMUs (garajes)
DMUs = df['Garage Name'].tolist()

# Definir inputs y outputs según tu archivo
inputs = [
    'Staff',
    'Show room space',
    'Population in category 1',
    'Population in category 2',
    'Enquiries Alpha model',
    'Enquiries Beta model'
]

outputs = [
    'Alpha sales',
    'Beta sales',
    'Profit'
]

# Crear diccionarios con los valores
x = {(i, dmu): df.loc[df['Garage Name'] == dmu, i].values[0] for i in inputs for dmu in DMUs}
y = {(r, dmu): df.loc[df['Garage Name'] == dmu, r].values[0] for r in outputs for dmu in DMUs}

# Guardar resultados
resultados = []

for k in DMUs:
    model = ConcreteModel()

    model.I = Set(initialize=inputs)
    model.R = Set(initialize=outputs)
    model.J = Set(initialize=DMUs)

    model.v = Var(model.I, domain=NonNegativeReals)  # pesos inputs
    model.u = Var(model.R, domain=NonNegativeReals)  # pesos outputs

    # Objetivo: maximizar eficiencia del garaje k
    model.obj = Objective(expr=sum(model.u[r] * y[r, k] for r in model.R), sense=maximize)

    # Normalización: suma ponderada de inputs del garaje k = 1
    model.norm = Constraint(expr=sum(model.v[i] * x[i, k] for i in model.I) == 1)

    # Restricción de eficiencia para todos los garajes
    def constraint_rule(model, j):
        return sum(model.u[r] * y[r, j] for r in model.R) - sum(model.v[i] * x[i, j] for i in model.I) <= 0

    model.constraints = Constraint(model.J, rule=constraint_rule)

    # Resolver con GLPK
    solver = SolverFactory('glpk')
    result = solver.solve(model, tee=False)

    # Obtener resultados
    eficiencia = value(model.obj)
    pesos_inputs = {i: value(model.v[i]) for i in inputs}
    pesos_outputs = {r: value(model.u[r]) for r in outputs}

    # Guardar todo
    resultados.append({
        'Garage': k,
        'Eficiencia': eficiencia,
        'Pesos Inputs': pesos_inputs,
        'Pesos Outputs': pesos_outputs
    })

# Mostrar resultados
for res in resultados:
    print(f"\n➡️ Garage: {res['Garage']}")
    print(f"   Eficiencia: {round(res['Eficiencia'], 4)}")
    print("   Pesos de Inputs:")
    for i, val in res['Pesos Inputs'].items():
        print(f"      {i}: {round(val, 4)}")
    print("   Pesos de Outputs:")
    for r, val in res['Pesos Outputs'].items():
        print(f"      {r}: {round(val, 4)}")


➡️ Garage: Winchester
   Eficiencia: 0.8354
   Pesos de Inputs:
      Staff: 0.0124
      Show room space: 0.0
      Population in category 1: 0.0
      Population in category 2: 0.0025
      Enquiries Alpha model: 0.0947
      Enquiries Beta model: 0.0195
   Pesos de Outputs:
      Alpha sales: 0.1132
      Beta sales: 0.0039
      Profit: 0.4044

➡️ Garage: Andover
   Eficiencia: 0.9174
   Pesos de Inputs:
      Staff: 0.1086
      Show room space: 0.0377
      Population in category 1: 0.0
      Population in category 2: 0.0
      Enquiries Alpha model: 0.0
      Enquiries Beta model: 0.0271
   Pesos de Outputs:
      Alpha sales: 0.3989
      Beta sales: 0.0
      Profit: 0.0

➡️ Garage: Basingstoke
   Eficiencia: 1.0
   Pesos de Inputs:
      Staff: 0.4286
      Show room space: 0.0
      Population in category 1: 0.0
      Population in category 2: 0.0
      Enquiries Alpha model: 0.0714
      Enquiries Beta model: 0.0
   Pesos de Outputs:
      Alpha sales: 1.25
      Beta sale

In [2]:
resultados

[{'Garage': 'Winchester',
  'Eficiencia': 0.8353963532226085,
  'Pesos Inputs': {'Staff': 0.0124265446060867,
   'Show room space': 0.0,
   'Population in category 1': 0.0,
   'Population in category 2': 0.002470272953191,
   'Enquiries Alpha model': 0.0947279610079496,
   'Enquiries Beta model': 0.0195458109378824},
  'Pesos Outputs': {'Alpha sales': 0.113198996497834,
   'Beta sales': 0.00387454031848345,
   'Profit': 0.404449090690567}},
 {'Garage': 'Andover',
  'Eficiencia': 0.9173789173789176,
  'Pesos Inputs': {'Staff': 0.108618233618234,
   'Show room space': 0.0377492877492877,
   'Population in category 1': 0.0,
   'Population in category 2': 0.0,
   'Enquiries Alpha model': 0.0,
   'Enquiries Beta model': 0.0270655270655272},
  'Pesos Outputs': {'Alpha sales': 0.398860398860399,
   'Beta sales': 0.0,
   'Profit': 0.0}},
 {'Garage': 'Basingstoke',
  'Eficiencia': 0.9999999999999991,
  'Pesos Inputs': {'Staff': 0.428571428571429,
   'Show room space': 0.0,
   'Population in cat

In [4]:
# Mostrar resultados
df_results = pd.DataFrame(resultados)
print(df_results)

         Garage  Eficiencia  \
0    Winchester    0.835396   
1       Andover    0.917379   
2   Basingstoke    1.000000   
3         Poole    0.864467   
4        Woking    0.845000   
5       Newbury    1.000000   
6    Portsmouth    1.000000   
7     Alresford    1.000000   
8     Salisbury    1.000000   
9     Guildford    0.802439   
10        Alton    1.000000   
11    Weybridge    0.854345   
12   Dorchester    0.866522   
13     Bridport    0.981572   
14     Weymouth    1.000000   
15     Portland    1.000000   
16   Chichester    0.825417   
17  Petersfield    1.000000   
18     Petworth    0.987836   
19     Midhurst    0.888808   
20      Reading    0.983967   
21  Southampton    1.000000   
22  Bournemouth    1.000000   
23       Henley    1.000000   
24   Maidenhead    1.000000   
25      Fareham    1.000000   
26       Romsey    1.000000   
27     Ringwood    0.907502   

                                         Pesos Inputs  \
0   {'Staff': 0.0124265446060867, 'Show roo

In [5]:
df_results

,Garage,Eficiencia,Pesos Inputs,Pesos Outputs
0,Winchester,0.835396,"{'Staff': 0.0124265446060867, 'Show room space...","{'Alpha sales': 0.113198996497834, 'Beta sales..."
1,Andover,0.917379,"{'Staff': 0.108618233618234, 'Show room space'...","{'Alpha sales': 0.398860398860399, 'Beta sales..."
2,Basingstoke,1.000000,"{'Staff': 0.428571428571429, 'Show room space'...","{'Alpha sales': 1.25, 'Beta sales': 0.0, 'Prof..."
3,Poole,0.864467,"{'Staff': 0.0, 'Show room space': 0.0188451284...","{'Alpha sales': 0.0651289289214115, 'Beta sale..."
4,Woking,0.845000,"{'Staff': 0.0150829089095783, 'Show room space...","{'Alpha sales': 0.0, 'Beta sales': 0.217669299..."
5,Newbury,1.000000,"{'Staff': 0.0, 'Show room space': 0.0, 'Popula...","{'Alpha sales': 0.125, 'Beta sales': 0.0, 'Pro..."
6,Portsmouth,1.000000,"{'Staff': 0.111125574937082, 'Show room space'...","{'Alpha sales': 0.0, 'Beta sales': 0.953744684..."
7,Alresford,1.000000,"{'Staff': 0.0185670226722319, 'Show room space...","{'Alpha sales': 0.0, 'Beta sales': 0.267950356..."
8,Salisbury,1.000000,"{'Staff': 0.159914017717561, 'Show room space'...","{'Alpha sales': 0.5, 'Beta sales': 0.0, 'Profi..."
9,Guildford,0.802439,"{'Staff': 0.0143328989864454, 'Show room space...","{'Alpha sales': 0.0619833804437647, 'Beta sale..."
